In [6]:
import requests
import sys
import pandas as pd
from pandas import json_normalize
import json

# Spider Project's API Server
url = 'http://89.106.173.60:62449/'
# Please prepend with the full path
filename = 'C:\\Users\\djumagalieve\\Documents\\Spider Project\\Demo Projects\\AD_Aksai_S.1054.sprj'
reportName = 'C:\\Users\\djumagalieve\\Documents\\Spider Project\\Demo Projects\\dsmDays.001.sdoc'

r = requests.post( url, json={'command':'login', 'user':'Dzhumaes', 'password':'ElS173tsm'} )
print('Login request done.') if r.status_code == 200 else sys.exit(0)
r = r.json()
if r['errcode'] != 0:
	print('Invalid login or password!')
	sys.exit(0)

# return docHandle.key
project = requests.post(url, json={'command': 'openFile', 'fileName': filename, 'sessId': r['sessId']})
print('\nProject opened.') if project.status_code == 200 else sys.exit(0)
docHandleProject = project.json()['docHandle']
if project['errcode'] != 0:
	print('Failed to open file!')
	sys.exit(0)

# execute Script
project = requests.post(url, json={"command":"executeScript","docHandle": docHandleProject,'scriptLibName': 'needDSMDays', 'sessId': r['sessId']})
print('\nScript executed.') if project.status_code == 200 else sys.exit(0)
if project['errcode'] != 0:
	print('Failed to execute Script!')
	sys.exit(0)
    
# return docHandle.key
report = requests.post(url, json={'command': 'openFile', 'fileName': reportName, 'sessId': r['sessId']})
print('\nReport opened.') if report.status_code == 200 else sys.exit(0)
docHandleReport = report.json()['docHandle']
if report['errcode'] != 0:
	print('Failed to open file!')
	sys.exit(0)

# return tableHandle.key
report = requests.post(url, json={'command':'getTableHandle','docHandle': docHandleReport,'sessId':r['sessId']})
print('\nTable handle obtained.') if report.status_code == 200 else sys.exit(0)
tableHandle = report.json()['tableHandle']
if report['errcode'] != 0:
	print('Failed to get tables handle!')
	sys.exit(0)

# return table
report = requests.post(url, json={'command':'getTable','tableHandle': str(tableHandle),'sessId':r['sessId']})
print('\nTable obtained.') if report.status_code == 200 else sys.exit(0)
data = report.json()['array']
if report['errcode'] != 0:
	print('Failed to obtain the table!')
	sys.exit(0)

data = str(data).replace("'",'"')
data = json.loads(data)

data = json_normalize(data)

# обработка NaN значений
data = data.fillna(0)
# замена типов данных
for i in data:
    if data[i].dtypes == 'float64':
        data[i] = data[i].astype('int64')
    else:
        continue
        
# Фреймы с данными за первые 20 дней месяца и последние 20 дней месяца
data_f20_days = data.iloc[:, 5:25]
data_l20_days = data.iloc[:, -20:]

# Серии со средними значениями по каждой строке
data_f20_days = data_f20_days.mean(axis=1)
data_l20_days = data_l20_days.mean(axis=1)

# конкатенация data_f20_days и data_l20_days по максимальному значению из 2 средних
data_max_mean = pd.concat([data_f20_days, data_l20_days], axis=1).max(axis=1)

# слияние в итоговый DataFrame (столбец ParName с типами техники + максимальное среднее)
fdata = data.iloc[:, 2:3].merge(data_max_mean.to_frame(), left_index=True, right_index=True)
fdata = fdata.rename(columns={0: 'MaxMean'})
fdata['MaxMean'] = fdata['MaxMean'].astype('int64')
fdata

Login request done.

Project opened.


TypeError: 'Response' object is not subscriptable